In [ ]:
#Komunikacja OPC UA pomiędzy serwerem (PC Station) a PLC S7-1500. 
#Serwer OPC realizowany poprzez SIMATIC NET v16 - PC Station. Sterownik S7-1511f fw v2.9. Programowany w TIA Portal v16
#Logowanie do serwera jako gość tzn. bez użycia certyfikatu, loginu i hasła
#Połączenie pomiędzy Serwerem OPC UA a PLC S7-1500 jest połączeniem typu S7. 
#Data Block i Tag w PLC są zoptymalizowane do połączenia z serwerem OPC UA, przez co używany jest serwer OPC.SIMATICNET.S7OPT
#
#Pobierane dane:
#       Array [0..2] of Byte
#       Bool
#       Byte - Hex
#       Char (znaki ASCII)
#       Date and Time - zmienna DTL z bloczka RD_SYS_T
#       Double word - Hex
#       Int
#       Real (wartości dziesiętne zmienno przecinkowe)
#       String (tekst)
#       Structura:
#           Bool
#           Byte
#
#Zmiana wartości INT w sterowniku PLC, incrementacja o 1 (jeden)
#
#Zapis do pliku CSV:
#   odczytanych danych z sterownika PLC
#   Zmiany wartości INT w sterowniku PLC oraz odczytu reszty danych
#   Oznaczenie w pliku CSV z jakiego sterownika zostały odczytane/zapisane dane - S7-1500, S7-1500, S7-300
#   Oznaczenie w pliku CSV czy dane zostały odczytane czy nadpisane w sterowniku PLC
#
#
#
#Wykorzystane dodatkowe biblioteki:
#OPC UA https://github.com/FreeOpcUa/python-opcua

In [ ]:
    #biblioteki do obsługi Client OPC i OPC UA
from opcua import Client, ua
    #obsługa zapisu do pliku CSV
import csv
    #sprawdzenie czy pliku .csv istnieje
import os

In [ ]:
    #połączenie do serwera bez dodatkowych poświadczeń jako gość (guest)
    #adres IP z portem serwera OPC UA, używany jest serwer OPC.SIMATICNET.S7OPT
url = 'opc.tcp://192.168.60.80:55105'
    #nawiązanie połączenia z serwerem
client = Client(url)
client.connect()
print(f'Połączono z: {url}')

In [ ]:
    #odczyt wartości Array [0..2] of Byte - odczytana została cała tablica 
zmienna_array0 = client.get_node('ns=6;s=S71500ET200MP station_1.1511f.data_db.zm_array').get_value()
    #wyświetlenie pojedyńczych wartości tablicy jako wartości szesnastkowe (HEX) 
    #zmiana zapisu wartości na string, aby umożliwić prosty zapis w pliku CSV oraz analizę tego pliku (wyodrębnienie pojedyńczych zmiennych)
zmienna_array_0 = str(hex(zmienna_array0[0]))
zmienna_array_1 = str(hex(zmienna_array0[1]))
zmienna_array_2 = str(hex(zmienna_array0[2]))
    #zapis zmienych z tablicy do jednej zmiennej, aby zapisać w jednej kolumnie w pliku CSV. Wartości oddzielonę są średnikiem ";"
zmienna_array = zmienna_array_0 + ';' + zmienna_array_1 + ';' + zmienna_array_2
    #wyświetlenie odczytanej zmiennej array (połączonych wartości)
print(zmienna_array)

In [ ]:
    #odczyt wartości Bool, wartość wyświetlana jest jako False lub True
zmienna_bool = client.get_node('ns=6;s=S71500ET200MP station_1.1511f.data_db.zm_bool').get_value()
    #wyświetlenie odczytanej zmiennej bool
print(zmienna_bool)

In [ ]:
    #odczyt wartości Byte jako wartość szesnastkowa (Hex) 
    #Wartości Byte w sterowniku PLC zapisane są jako wartości Hexadecymalne
zmienna_byte = hex(client.get_node('ns=6;s=S71500ET200MP station_1.1511f.data_db.zm_byte').get_value())
    #wyświetlenie odczytanej zmiennej byte
print(zmienna_byte)

In [ ]:
    #odczyt wartosci decymalnej z zmiennej char - wartość kodu ASCII
    #sterownik PLC umożliwia również zastosowanie rozszerzonego kodu ASCII 
        #więcej informacji o możliwych zastosowaniach kodów ASCII w PLC S7-1x00 https://support.industry.siemens.com/cs/mdm/109741593?c=79989723403&lc=en-CO
zmienna_char = client.get_node('ns=6;s=S71500ET200MP station_1.1511f.data_db.zm_char').get_value()
    #zmiana wartosci decymalnej na znak z tablicy ASCII
zmienna_char = chr(zmienna_char)
    #wyświetlenie odczytanej zmiennej char
print(zmienna_char)

In [ ]:
    #odczyt całej zmiennej DTL (Date and Time)
zmienna_date_time = client.get_node('ns=6;s=S71500ET200MP station_1.1511f.data_db.zm_datetime_dtl').get_value()
print(zmienna_date_time)

    #odczyt pojedyńczych wartości z tablicy DTL
    #odczyt dnia, miesiąca i roku
zmienna_date_time_day = client.get_node('ns=6;s=S71500ET200MP station_1.1511f.data_db.zm_datetime_dtl.DAY').get_value()
zmienna_date_time_month = client.get_node('ns=6;s=S71500ET200MP station_1.1511f.data_db.zm_datetime_dtl.MONTH').get_value()
zmienna_date_time_year = client.get_node('ns=6;s=S71500ET200MP station_1.1511f.data_db.zm_datetime_dtl.YEAR').get_value()
    #odczyt godziny, minuty, sekudny, nanosekundy 
zmienna_date_time_hour = client.get_node('ns=6;s=S71500ET200MP station_1.1511f.data_db.zm_datetime_dtl.HOUR').get_value()
zmienna_date_time_minute = client.get_node('ns=6;s=S71500ET200MP station_1.1511f.data_db.zm_datetime_dtl.MINUTE').get_value()
zmienna_date_time_second = client.get_node('ns=6;s=S71500ET200MP station_1.1511f.data_db.zm_datetime_dtl.SECOND').get_value()
zmienna_date_time_nanosecond = client.get_node('ns=6;s=S71500ET200MP station_1.1511f.data_db.zm_datetime_dtl.NANOSECOND').get_value()
    #odczyt dnia tygodnia 
zmienna_date_time_weekday = client.get_node('ns=6;s=S71500ET200MP station_1.1511f.data_db.zm_datetime_dtl.WEEKDAY').get_value() 
    
    #zmiana zapisu wartości na string, aby umożliwić prosty zapis w pliku CSV
    #zmiana z wartości INT na String
zmienna_date_time_day = str(zmienna_date_time_day)
zmienna_date_time_month = str(zmienna_date_time_month)
zmienna_date_time_year = str(zmienna_date_time_year)
zmienna_date_time_hour = str(zmienna_date_time_hour)
zmienna_date_time_minute = str(zmienna_date_time_minute)
zmienna_date_time_second = str(zmienna_date_time_second)
zmienna_date_time_weekday = str(zmienna_date_time_weekday)

    #zapis zmienych z tablicy do jednej zmiennej, aby zapisać w jednej kolumnie w pliku CSV. Wartości oddzielonę są krpką "."
zmienna_date_time_date = zmienna_date_time_year + '.' + zmienna_date_time_month + '.' + zmienna_date_time_day
    #wyświetlenie odczytanej zmiennej Data
print("Data: ",zmienna_date_time_date)
        #zapis zmienych z tablicy do jednej zmiennej, aby zapisać w jednej kolumnie w pliku CSV. Wartości oddzielonę są dwukropkiem ":"
zmienna_date_time_time = zmienna_date_time_hour + ':' + zmienna_date_time_minute + ':' + zmienna_date_time_second
    #wyświetlenie odczytanej zmiennej czas
print("Godzina: ",zmienna_date_time_time)
    #wyświetlenie odczytanej zmiennej dzień tygodnia
print("Dzien tygodnia: ", zmienna_date_time_weekday)


In [ ]:
    #odczyt wartości zmiennej Double Word jako wartość szesnastkową (Hex). Tak samo odcztuje się wartość Word.
    #Wartości Word i Double Word w sterowniku PLC zapisane są jako wartości Hexadecymalne
zmienna_double_word = hex(client.get_node('ns=6;s=S71500ET200MP station_1.1511f.data_db.zm_dword').get_value())
    #wyświetlenie odczytanej zmiennej Double Word
print(zmienna_double_word)

In [ ]:
    #odczytanie wartości Int
zmienna_int = client.get_node('ns=6;s=S71500ET200MP station_1.1511f.data_db.zm_int').get_value()
    #wyświetlenie odczytanej zmiennej Int
print(zmienna_int)

In [ ]:
    #odczytanie wartości Real ze sterownika PLC. Wartość Real jest wartością zmienno przecinkową
zmienna_real = float(client.get_node('ns=6;s=S71500ET200MP station_1.1511f.data_db.zm_real').get_value())
    #zaokrąglenie odczytanej wartości do 2 miejsc po przecinku
zmienna_real = round(zmienna_real,2)
    #wyświetlenie odczytanej zmiennej Real
print(zmienna_real)

In [ ]:
    #odczytanie wartości String, która jest tekstowym typem danych 
zmienna_string = client.get_node('ns=6;s=S71500ET200MP station_1.1511f.data_db.zm_string').get_value()
    #więcej informacji o możliwych zastosowaniach dłuższych łańcuchów wartości jednowyrazowych (dwubajtowych - zastosowanie WString) w PLC S7-1x00
    #https://support.industry.siemens.com/cs/mdm/109741593?c=79989723403&lc=en-CO
    #wyświetlenie odczytanej zmiennej String
print(zmienna_string)

In [ ]:
    #odczyt danych z struktury - data type
    #odczyt wartości bool z struktury
zmienna_structura_bool = client.get_node('ns=6;s=S71500ET200MP station_1.1511f.data_db.zm_struct.zm_struct_bool').get_value()
    #odczyt wartości byte z struktury. Wartość odczytywana jest jako wartość szesnastkowa 
zmienna_structura_byte = hex(client.get_node('ns=6;s=S71500ET200MP station_1.1511f.data_db.zm_struct.zm_struct_byte').get_value())

    #zmiana zapisu wartości na string, aby umożliwić prosty zapis w pliku CSV oraz analizę tego pliku (wyodrębnienie pojedyńczych zmiennych)
zmienna_structura_bool_0 = str(zmienna_structura_bool)
zmienna_structura_byte_1 = str(zmienna_structura_byte)
        #zapis zmienych z tablicy do jednej zmiennej, aby zapisać w jednej kolumnie w pliku CSV. Wartości oddzielonę są średnikiem ";"                       
zmienna_structura = zmienna_structura_bool_0 + ';' + zmienna_structura_byte_1
    #wyświetlenie zmiennej struktura
print(zmienna_structura)

In [ ]:
    #z jakiego sterownika są zapisywane dane do pliku CSV
sterownik = 'S7-1511f'

    #przypisanie odpowiedniej wartości dla kolumny Odczyt/zapis
odczyt_zapis = 'odczyt'

    #zapis odczytanych danych do pliku csv
    #sprawdzenie czy istnieje plik o nazwie "opc.csv"
if(os.path.isfile('opc.csv')): #jeżeli tak to dopisz
        with open('opc.csv','a',  encoding='UTF8', newline='') as file: #('nazwa pliku','a - dopisz do pliku','kodowanie','do nowej linii')
            naglowek = ['Data','godzina','array','bool','byte','char','double word','int','real','string','structura','odczyt/zapis','sterownik'] #nagłówek pliku CSV
            writer = csv.DictWriter(file, delimiter =';', fieldnames=naglowek) #dopisz do pliku (plik,'separacja','naglowek')
            writer.writerow({'Data':zmienna_date_time_date,'godzina':zmienna_date_time_time,
                        'array':zmienna_array,'bool':zmienna_bool,'byte':zmienna_byte,
                        'char':zmienna_char,'double word':zmienna_double_word,'int':zmienna_int,'real':zmienna_real,'string':zmienna_string,
                        'structura':zmienna_structura,'odczyt/zapis':odczyt_zapis,'sterownik':sterownik}) #dopisanie do pliku wartości zmiennych 
        
else: #inaczej stworz plik z naglowkami
        with open('opc.csv','x',  encoding='UTF8', newline='') as file: #('nazwa pliku','a - dopisz do pliku','kodowanie','do nowej linii')
            naglowek = ['Data','godzina','array','bool','byte','char','double word','int','real','string','structura','odczyt/zapis','sterownik'] #nagłówek pliku CSV
            writer = csv.DictWriter(file, delimiter =';', fieldnames=naglowek) #dopisz do pliku (plik,'separacja','naglowek')
            writer.writeheader() #dopisz do pliku nagłówek - co nie dzieje się przy dopisywaniu do pliku,
            #ponieważ w innym przypadku za każdym razem w pliku CSV zapis byłby (wiersze oddzielone średnikiem): nagłówek, zmienne, nagłówek, zmienne,...
            writer.writerow({'Data':zmienna_date_time_date,'godzina':zmienna_date_time_time,
                        'array':zmienna_array,'bool':zmienna_bool,'byte':zmienna_byte,
                        'char':zmienna_char,'double word':zmienna_double_word,'int':zmienna_int,'real':zmienna_real,'string':zmienna_string,
                        'structura':zmienna_structura,'odczyt/zapis':odczyt_zapis,'sterownik':sterownik}) #dopisanie do pliku wartości zmiennych

In [ ]:
    #przypisanie odpowiedniej wartości dla kolumny Odczyt/zapis
odczyt_zapis = 'zapis'

    #zmiana wartości INT aby wprowadzić nową zmienną do sterownika PLC
    #odczyt zmiennej INT
zmienna_set = client.get_node('ns=6;s=S71500ET200MP station_1.1511f.data_db.zm_int')
    #oddczyt wartości zmiennej INT
wartosc = client.get_node('ns=6;s=S71500ET200MP station_1.1511f.data_db.zm_int').get_value()
    #incrementacja wartości (+1)
wartosc += 1
    #nadpisanie wartości do sterownika PLC
zmienna_set_dv = ua.DataValue(ua.Variant(wartosc, ua.VariantType.Int16))
zmienna_set.set_value(zmienna_set_dv)

    #ponowny odczyt zmiennej Int, aby zapisać poprawną wartośc do pliku csv
zmienna_int = client.get_node('ns=6;s=S71500ET200MP station_1.1511f.data_db.zm_int').get_value()
    #zapis do pliku csv, nie jest sprawdzany czy plik istnieje, ponieważ istnieje lub został stworzony w poprzednim kroku
with open('opc.csv','a',  encoding='UTF8', newline='') as file:
        naglowek = ['Data','godzina','array','bool','byte','char','double word','int','real','string','structura','odczyt/zapis','sterownik']
        writer = csv.DictWriter(file, delimiter =';', fieldnames=naglowek)        
        writer.writerow({'Data':zmienna_date_time_date,'godzina':zmienna_date_time_time,
                        'array':zmienna_array,'bool':zmienna_bool,'byte':zmienna_byte,
                        'char':zmienna_char,'double word':zmienna_double_word,'int':zmienna_int,'real':zmienna_real,'string':zmienna_string,
                        'structura':zmienna_structura,'odczyt/zapis':odczyt_zapis,'sterownik':sterownik})

In [ ]:
    #rozłączanie z serwerem OPC UA
client.disconnect()
print('Rozłączono z: {url}')